In [35]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [36]:
## Load the trained model, scaler pickle, onehot
model = load_model('model.h5')

## load the encoder and scaler
with open('onehot_encoder_geo.pkl', 'rb') as file:
    label_encoder_geo = pickle.load(file)
    
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)
    
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [37]:
## Example input data
input_data = {
    'CreditScore': 650,
    'Age': 30,
    'Gender': 'Male',
    'Geography': 'France',
    'Tenure': 5,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
    }

In [38]:
## One-hot encode categorical features
from sklearn.preprocessing import OneHotEncoder
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))

e:\ANN_CLASSIFICATION\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [39]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Age,Gender,Geography,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,650,30,Male,France,5,60000,2,1,1,50000


In [40]:
## Encode categorical features
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Age,Gender,Geography,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,650,30,1,France,5,60000,2,1,1,50000


In [42]:
## Combine one-hot encoded 
input_df = pd.concat([input_df.drop('Geography', axis=1),geo_encoded_df],axis=1)
input_df


,CreditScore,Age,Gender,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,650,30,1,5,60000,2,1,1,50000,1.0,0.0,0.0


Align columns before transforming

In [46]:
print(scaler.feature_names_in_)


['CreditScore' 'Gender' 'Age' 'Tenure' 'Balance' 'NumOfProducts'
 'HasCrCard' 'IsActiveMember' 'EstimatedSalary' 'Geography_France'
 'Geography_Germany' 'Geography_Spain']


In [47]:
print(input_df.columns.tolist())


['CreditScore', 'Age', 'Gender', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Geography_France', 'Geography_Germany', 'Geography_Spain']


In [48]:
input_df = input_df.reindex(columns=scaler.feature_names_in_)


In [52]:
## Scaling the input data
input_scaled = scaler.transform(input_df)
input_scaled


array([[-0.01709861,  0.91324755, -0.84593077, -0.00134472, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [55]:
## Predict using the trained mode
# l
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step


array([[0.01125401]], dtype=float32)

In [56]:
prediction_proba = prediction[0][0]

In [57]:
prediction_proba

np.float32(0.011254008)

In [58]:
if prediction_proba > 0.5:
    print("The customer is likely to churn.")   
else:
    print("The customer is not likely to churn.")

The customer is not likely to churn.
